In [1]:
import warnings
warnings.filterwarnings('once')

# To Do
* read data 
* process
    * remove html tag using bs4
    * clean numbers and punctuations, lower case everything
    * tokenize
    * remove stop words
* bag of words
    * vectorise 
* fit a model 
    * read test and preprocess
    * fit random forest 
    * predict



In [117]:
%%time
from bs4 import BeautifulSoup             
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
import csv

CPU times: user 23 µs, sys: 1e+03 ns, total: 24 µs
Wall time: 27.2 µs


In [15]:
nltk.download() 

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [19]:
raw_train = pd.read_csv("./labeledTrainData.tsv",delimiter='\t',quoting=3,header=0)

In [43]:
# Processing function
def clean(review):
    clean_review =  BeautifulSoup(review).get_text()
    only_letters = re.sub("[^a-zA-Z]"," ",clean_review)
    lower=only_letters.lower().split()
    words = [w for w in lower if not w in stopwords.words("English")]
    return( " ".join(words))  
print(clean(raw_train.iloc[0][2]))

stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate working

In [50]:
raw_train.iloc[0][2]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [52]:
clean_reviews=pd.DataFrame(columns=["id","review"])
for i in range(len(raw_train)):
    clean_reviews.loc[len(clean_reviews)]= clean(raw_train.iloc[i][2])
    if i%1000==0:
        print(i," ")

0  
1000  
2000  
3000  
4000  
5000  
6000  
7000  
8000  
9000  
10000  
11000  
12000  
13000  
14000  
15000  
16000  
17000  
18000  
19000  
20000  
21000  
22000  
23000  
24000  


In [72]:
backup=clean_reviews


review    superbly trashy wondrously unpretentious explo...
Name: 4, dtype: object


In [88]:
backup1=backup.drop(['id'],axis=1)
reviews=[]
print(backup1.iloc[0])

review    stuff going moment mj started listening music ...
Name: 0, dtype: object


In [89]:
backup=backup.drop(['id'],axis=1)
reviews=[]
for i in range(len(backup)):
    reviews.append(backup.iloc[i][0])
print(len(reviews))

25000


In [90]:
vectorizer = CountVectorizer(analyzer = "word",tokenizer = None, preprocessor = None,\
                             stop_words = None,max_features = 5000) 
train_data_features = vectorizer.fit_transform(reviews)
print(train_data_features.shape)


(25000, 5000)


In [102]:
%%time
forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit( train_data_features, raw_train["sentiment"] )

In [107]:
test=pd.read_csv('./testData.tsv',delimiter='\t',quoting=3, header=0)
print(test.columns)
test_clean_reviews = []
for i in range(len(test)):
    if i%1000==0:
        print(i)
    test_clean_reviews.append(clean(test.iloc[i]['review']))
print(len(test_clean_reviews))

Index(['id', 'review'], dtype='object')
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000


In [112]:
test_data_features = vectorizer.fit_transform(test_clean_reviews)
predictions= forest.predict(test_data_features)
result=pd.DataFrame(columns=['id','sentiment'])
result['id']=test['id']
result['sentiment']=predictions
print(result.shape)

(25000, 2)


In [124]:
x="ki"
print(type(x))

<class 'str'>


In [129]:
result.to_csv("output.csv",sep=',',index=False,quoting=csv.QUOTE_NONE)

In [130]:
temp=pd.read_csv("output.csv")
temp.to_csv("outpu1.csv",sep=',',index=False,quoting=csv.QUOTE_NONE)